# Lab — Label Errors

This lab highlights data-centric AI techniques (using [confident learning](https://jair.org/index.php/jair/article/view/12125)) to improve the accuracy of an XGBoost classifier on a noisy dataset that has label errors.

The DCAI techniques demonstrated in this lab involve optimizing the dataset itself rather than altering the model's architecture or hyperparameters. As a result, it is possible to achieve further improvements in accuracy by fine-tuning the model in conjunction with the newly enhanced data, but that is not the focus of this lab.

In this lab, we will:

- Establish a baseline [XGBoost](https://xgboost.readthedocs.io/) model accuracy on the original data
- Automatically find mislabeled data points by:
    - Computing out-of-sample predicted probabilities
    - Estimating the number of label errors using confident learning
    - Ranking errors, using the number of label errors as a cutoff in identifying issues
- Remove the bad data
- Retrain the exact same XGBoost model to see the improvement in test accuracy

## Software installation

This lab relies on a couple PyPI packages. If you don't have them installed, run the following cell:

In [1]:
!pip install xgboost==1.7 scikit-learn pandas cleanlab

## Setup and Data Processing

Let's take a look at the dataset used in this lab, a tabular dataset of student grades.

The data includes three exam scores (numerical features), a written note (categorical feature with missing values), and a (noisy) letter grade (categorical label). Our aim is to train a model to classify the grade for each student based on the other features.

In this dataset, 20% of the grade labels are actually incorrect (the `noisy_letter_grade` column). Synthetic noise was added to this dataset for the purpose of this lab. In this lab, we have access to the true letter grade each student should have received (the `letter_grade` column), which we use for evaluating both the underlying accuracy of model predictions and how well our approach detects which data are mislabeled. We are careful to only use these true grades for evaluation, not for model training.

In the real world, you don't have access to the true labels (you only observe the `noisy_letter_grade`, not the true `letter_grade`). So when evaluating models in the real world, you have to be careful to make sure that your test set is free of error (using methods like those covered in this lab, ideally combined with human review).

In [2]:
from xgboost import XGBClassifier
from sklearn import preprocessing
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np

df = pd.read_csv("./student-grades.csv")
df.head()

,stud_ID,exam_1,exam_2,exam_3,notes,letter_grade,noisy_letter_grade
0,f48f73,53,77,93,NaN,C,C
1,0bd4e7,81,64,80,great participation +10,B,B
2,e1795d,74,88,97,NaN,B,B
3,cb9d7a,61,94,78,NaN,C,C
4,9acca4,48,90,91,NaN,C,C


In [3]:
df_c = df.copy()
# Transform letter grades and notes to categorical numbers.
# Necessary for XGBoost.
df['letter_grade'] = preprocessing.LabelEncoder().fit_transform(df['letter_grade'])
df['noisy_letter_grade'] = preprocessing.LabelEncoder().fit_transform(df['noisy_letter_grade'])
df['notes'] = preprocessing.LabelEncoder().fit_transform(df["notes"])
df['notes'] = df['notes'].astype('category')
df.head()

,stud_ID,exam_1,exam_2,exam_3,notes,letter_grade,noisy_letter_grade
0,f48f73,53,77,93,5,2,2
1,0bd4e7,81,64,80,2,1,1
2,e1795d,74,88,97,5,1,1
3,cb9d7a,61,94,78,5,2,2
4,9acca4,48,90,91,5,2,2


# Get What We Need

To apply confident learning (the technique explained in today's lecture), we need to obtain [**out-of-sample** predicted probabilities](https://docs.cleanlab.ai/stable/tutorials/pred_probs_cross_val.html#out-of-sample-predicted-probabilities) for all of our data. To do this, we can use K-fold cross validation: for each fold, we will train on some subset of our data and get predictions on the rest of the data that was _not_ used for training.

We need to choose a model in order to do this. For this lab, we'll use [XGBoost](https://xgboost.readthedocs.io/), a library implementing gradient-boosted decision trees, a class of model commonly used for tabular data.

In [4]:
# Prepare training data (remove labels from the dataframe) and labels
data = df.drop(['stud_ID', 'letter_grade', 'noisy_letter_grade'], axis=1)
labels = df['noisy_letter_grade']

# XGBoost(experimental) supports categorical data.
# Here we use default hyperparameters for simplicity.
# Get out-of-sample predicted probabilities and check model accuracy.
model = XGBClassifier(tree_method="hist", enable_categorical=True)

# Exercise 1: getting out-of-sample predicted probabilities

Compute out-of-sample predicted probabilities for every data point. You can do this manually using for loops and multiple invocations of model training and prediction, or you can use scikit-learn's [cross_val_predict](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html) (if you're using this function, take a look at the documentations, and in particular, the `method=` keyword argument).

In [5]:
# pred_probs should be a Nx5 matrix of out-of-sample predicted probabilities, with N = len(data)
from sklearn.model_selection import cross_val_predict


pred_probs = cross_val_predict(model, data, labels, method='predict_proba')

In [6]:
data.shape

(941, 4)

In [7]:
pred_probs.shape

(941, 5)

In [8]:
pred_probs

array([[9.9915704e-03, 1.0754441e-02, 9.5985240e-01, 5.0153048e-03,
        1.4386256e-02],
       [4.8501577e-02, 8.6473954e-01, 1.3487166e-03, 8.9849411e-03,
        7.6425239e-02],
       [7.7535305e-03, 9.5749086e-01, 1.9698440e-04, 9.6462184e-04,
        3.3594064e-02],
       ...,
       [1.8168031e-01, 4.3267220e-01, 4.6972525e-03, 3.4395212e-01,
        3.6998145e-02],
       [3.5881605e-02, 9.0193230e-01, 1.5249980e-02, 1.4933412e-03,
        4.5442801e-02],
       [8.2281864e-01, 2.2477485e-02, 1.1010921e-01, 2.8254291e-02,
        1.6340349e-02]], dtype=float32)

In [9]:
pred_probs[:,0].sum()

228.43729

## Checking model accuracy on original data

Now that we have out-of-sample predicted probabilities, we can also check the model's (cross-val) accuracy on the original (noisy) data, so we'll have a baseline to compare our final results.

In [10]:
preds = np.argmax(pred_probs, axis=1)
acc_original = accuracy_score(preds, labels)
print(f"Accuracy with original data: {round(acc_original*100,1)}%")

Accuracy with original data: 67.4%


# Finding label issues automatically

We count label issues using confident learning. First, we need to compute class thresholds for the different classes.

# Exercise 2: computing class thresholds

Implement the Confident Learning algorithm for computing class thresholds for the 5 classes. You can refer to slide 26 from today's lecture or see equation 2 in [this paper](https://jair.org/index.php/jair/article/view/12125).

The class threshold for each class is the model's expected (average) self-confidence for each class. In other words, to compute the threshold for a particular class, you can average the predicted probability for that class, for all datapoints that are labeled with that particular class.

In [11]:
def compute_class_thresholds(pred_probs: np.ndarray, labels: np.ndarray) -> np.ndarray:
    classes = np.unique(labels)
    
    t_j = np.zeros(5)
    
    for class_ in classes:
        sum_probs = pred_probs[(labels == class_),class_].sum()
        count = (labels == class_).sum()
        
        t_j[class_] = sum_probs/count
        
    return t_j
    

In [12]:
# should be a numpy array of length 5
thresholds = compute_class_thresholds(pred_probs, labels.to_numpy())

In [13]:
thresholds

array([0.72064711, 0.64154053, 0.67945671, 0.47879471, 0.45895547])

# Exercise 3: constructing the confident joint

Next, we compute the confident joint, a matrix that counts the number of label errors for each noisy label $\tilde{y}$ and true label $y^*$. You can follow the algorithm that we walked through in slide 27 from today's lecture, or see equation 1 in [this paper](https://jair.org/index.php/jair/article/view/12125).

The confident joint C is a K x K matrix (with K = 5 for this dataset), where `C[i][j]` is an estimate of the count of the number of data points with noisy label `i` and true label `j`. From lecture, recall that we put a data point in bin `(i, j)` if its given label is `i`, and its predicted probability for class `j` is above the threshold for class `j` (`thresholds[j]`). Each data point should only go in a single bin; if a data point's predicted probability is above the class threshold for multiple classes, it goes in the bin for which it has the highest predicted probability.

In [14]:
def compute_confident_joint(pred_probs: np.ndarray, labels: np.ndarray, thresholds: np.ndarray) -> np.ndarray:
    K = len(np.unique(labels))
    C = np.zeros((K, K))
    
    for i, pred  in enumerate(pred_probs):
        idx = None
        for j,prob in enumerate(pred):
            if prob >= thresholds[j]:
                if (idx and prob > pred[idx[1]]) or not idx:
                    idx = (labels[i], j)
        if idx: 
            C[idx] += 1

    return C

In [15]:
C = compute_confident_joint(pred_probs, labels.to_numpy(), thresholds)

In [16]:
C

array([[157.,   4.,  18.,   8.,  14.],
       [  4., 145.,   1.,  42.,  11.],
       [  2.,   4., 103.,   5.,   8.],
       [  3.,  37.,   0.,  76.,  10.],
       [ 19.,  16.,  16.,   8.,  70.]])

# Exercise 4: count the number of label issues

Now that we have the confident joint C, we can count the estimated number of label issues in our dataset. Recall that this is the sum of the off-diagonal entries (the cases where we estimate that a label has been flipped).

In [17]:
num_label_issues = np.sum(C) - np.trace(C)

In [18]:
print('Estimated noise rate: {:.1f}%'.format(100*num_label_issues / pred_probs.shape[0]))

Estimated noise rate: 24.4%


# Exercise 5: filter out label issues

In this lab, our approach to identifying issues is to rank the data points by a score ("self-confidence", the model's predicted probability for a data point's given label) and then take the top `num_label_issues` of those.

First, we want to compute the model's _self-confidence_ for each data point. For a data point `i`, that is `pred_probs[i, labels[i]]`.

In [19]:
# this should be a numpy array of length 941 of probabilities
self_confidences = [pred_probs[i, labels[i]] for i in range(len(labels))]# YOUR CODE HERE

Next, we rank the _indices_ of the data points by the self-confidence.

In [20]:
len(self_confidences)

941

In [29]:
# this should be a numpy array of length 941 of integer indices
ranked_indices = np.array(self_confidences) # YOUR CODE HERE

In [30]:
ranked_indices = np.argsort(ranked_indices)

In [ ]:
ranked_indices

Finally, let's compute the indices of label issues as the top `num_label_issues` items in the `ranked_indices`.

In [23]:
ranked_indices.shape

(941,)

In [24]:
issue_idx = ranked_indices[:int(num_label_issues)] # YOUR CODE HERE

In [25]:
issue_idx

array([7.78657923e-05, 2.81155662e-04, 2.87546456e-04, 3.21800064e-04,
       4.13272181e-04, 4.44249046e-04, 5.12100640e-04, 5.57839056e-04,
       6.27434289e-04, 7.04711012e-04, 8.75504978e-04, 8.79798783e-04,
       1.09987333e-03, 1.13028067e-03, 1.17013685e-03, 1.18901313e-03,
       1.23676739e-03, 1.30755617e-03, 1.33167207e-03, 1.44460122e-03,
       1.45960541e-03, 1.61577633e-03, 1.63124001e-03, 1.80985767e-03,
       1.93282578e-03, 2.37767259e-03, 2.45622848e-03, 2.67440616e-03,
       2.75136041e-03, 2.83866143e-03, 2.93544983e-03, 3.04869539e-03,
       3.42031918e-03, 3.45510920e-03, 3.56282433e-03, 3.61398445e-03,
       3.84316524e-03, 4.06277459e-03, 4.19223960e-03, 4.27698065e-03,
       4.32497449e-03, 4.49045002e-03, 4.64583328e-03, 4.65151248e-03,
       4.83085122e-03, 5.24225505e-03, 5.53090684e-03, 5.76075399e-03,
       5.77720720e-03, 5.78107312e-03, 6.31294260e-03, 6.39841333e-03,
       6.51997654e-03, 6.90162182e-03, 6.92856684e-03, 7.84466229e-03,
      

Let's look at a couple of the highest-ranked data points (most likely to be label issues):

In [26]:
df_c.iloc[ranked_indices[:5]]

,stud_ID,exam_1,exam_2,exam_3,notes,letter_grade,noisy_letter_grade
0,f48f73,53,77,93,NaN,C,C
0,f48f73,53,77,93,NaN,C,C
0,f48f73,53,77,93,NaN,C,C
0,f48f73,53,77,93,NaN,C,C
0,f48f73,53,77,93,NaN,C,C


# How'd We Do?

Let's go a step further and see how we did at automatically identifying which data points are mislabeled. If we take the intersection of the labels errors identified by Confident Learning and the true label errors, we see that our approach was able to identify 75% of the label errors correctly (based on predictions from a model that is only 67% accurate). 

In [27]:
# Computing percentage of true errors identified. 
true_error_idx = df[df.letter_grade != df.noisy_letter_grade].index.values
cl_acc = len(set(true_error_idx).intersection(set(issue_idx)))/len(true_error_idx)
print(f"Percentage of errors found: {round(cl_acc*100,1)}%")

Percentage of errors found: 0.0%


# Train a More Robust Model

Now that we have the indices of potential label errors within our data, let's remove them from our data, retrain our model, and see what improvement we can gain.

Keep in mind that our baseline model from above, trained on the original data using the `noisy_letter_grade` as the prediction label, achieved a cross-validation accuracy of 67%.

Let's use a very simple method to handle these label errors and just drop them entirely from the data and retrain our exact same `XGBClassifier`. In a real-world application, a better approach might be to have humans review the issues and _correct_ the labels rather than dropping the data points.

In [28]:
# Remove the label errors found by Confident Learning
data = df.drop(issue_idx)
clean_labels = data['noisy_letter_grade']
data = data.drop(['stud_ID', 'letter_grade', 'noisy_letter_grade'], axis=1)

# Train a more robust classifier with less erroneous data
model = XGBClassifier(tree_method="hist", enable_categorical=True)
clean_pred_probs = cross_val_predict(model, data, clean_labels, method='predict_proba')
clean_preds = np.argmax(clean_pred_probs, axis=1)

acc_clean = accuracy_score(clean_preds, clean_labels)
print(f"Accuracy with original data: {round(acc_original*100, 1)}%")
print(f"Accuracy with errors found by Confident Learning removed: {round(acc_clean*100, 1)}%")

# Compute reduction in error.
err = ((1-acc_original)-(1-acc_clean))/(1-acc_original)
print(f"Reduction in error: {round(err*100,1)}%")

KeyError: '[7.786579e-05, 0.00028115566, 0.00028754646, 0.00032180006, 0.00041327218, 0.00044424905, 0.00051210064, 0.00055783906, 0.0006274343, 0.000704711, 0.000875505, 0.0008797988, 0.0010998733, 0.0011302807, 0.0011701368, 0.0011890131, 0.0012367674, 0.0013075562, 0.0013316721, 0.0014446012, 0.0014596054, 0.0016157763, 0.00163124, 0.0018098577, 0.0019328258, 0.0023776726, 0.0024562285, 0.0026744062, 0.0027513604, 0.0028386614, 0.0029354498, 0.0030486954, 0.0034203192, 0.0034551092, 0.0035628243, 0.0036139844, 0.0038431652, 0.0040627746, 0.0041922396, 0.0042769806, 0.0043249745, 0.00449045, 0.0046458333, 0.0046515125, 0.004830851, 0.005242255, 0.005530907, 0.005760754, 0.005777207, 0.005781073, 0.0063129426, 0.0063984133, 0.0065199765, 0.006901622, 0.006928567, 0.007844662, 0.008037647, 0.0083276285, 0.008903196, 0.008909289, 0.009165551, 0.009166182, 0.010512229, 0.010570079, 0.010995868, 0.011294353, 0.01132341, 0.011646841, 0.011781578, 0.011972804, 0.012364553, 0.012442607, 0.01

After removing the suspected label issues, our model's new cross-validation accuracy is now 90%, which means we **reduced the error-rate of the model by 70%** (the original model had 67% accuracy). 

**Note: throughout this entire process we never changed any code related to model architecture/hyperparameters, training, or data preprocessing!  This improvement is strictly coming from increasing the quality of our data which leaves additional room for additional optimizations on the modeling side.**

# Conclusion

For the student grades dataset, we found that simply dropping identified label errors and retraining the model resulted in a 70% reduction in prediction error on our classification problem (with accuracy improving from 67% to 90%).

An implementation of the Confident Learning algorithm (and much more) is available in the [cleanlab](https://github.com/cleanlab/cleanlab) library on GitHub. This is how today's lab assignment can be done in a single line of code with Cleanlab:

In [ ]:
import cleanlab

cl_issue_idx = cleanlab.filter.find_label_issues(labels, pred_probs, return_indices_ranked_by='self_confidence')

In [ ]:
df_c.iloc[cl_issue_idx[:5]]

_Advanced topic_: you might notice that the above `cl_issue_idx` differs in length (by a little bit) from our `issue_idx`. The reason for this is that we implemented a slightly simplified version of the algorithm in this lab. We skipped a calibration step after computing the confident joint that makes the confident joint have the true noisy prior $p(labels)$ (summed over columns for each row) and also add up to the total number of examples. If you're interested in the details of this, see equation 3 and the subsequent explanation in the [paper](https://jair.org/index.php/jair/article/view/12125).